In [12]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import datetime as dt
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
data = pd.read_csv("data.csv")

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
data.head()

,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),Energy (kWh),GHG Savings (kg),Gasoline Savings (gallons),Port Type,Port Number,Plug Type,EVSE ID,Address 1,City,State/Province,Postal Code,Country,Latitude,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,6.249457,2.625,0.784,Level 2,2,J1772,NaN,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444572,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,0.106588,0.045,0.013,Level 2,2,J1772,NaN,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444572,-122.160309,USD,0.0,Customer,4,94301.0,4169,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,14.951777,6.280,1.876,Level 2,2,J1772,NaN,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444572,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,7.159643,3.007,0.899,Level 2,2,J1772,NaN,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444572,-122.160309,USD,0.0,Customer,6,94302.0,2545,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,1.957765,0.822,0.246,Level 2,2,J1772,NaN,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444572,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765,NaN,NaN,NaN


In [15]:
#replace white space in column names with _, strip ending white space

data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.rstrip()

In [16]:
data.dtypes

Station_Name                        object
MAC_Address                         object
Org_Name                            object
Start_Date                          object
Start_Time_Zone                     object
End_Date                            object
End_Time_Zone                       object
Transaction_Date_(Pacific_Time)     object
Total_Duration_(hh:mm:ss)           object
Charging_Time_(hh:mm:ss)            object
Energy_(kWh)                       float64
GHG_Savings_(kg)                   float64
Gasoline_Savings_(gallons)         float64
Port_Type                           object
Port_Number                          int64
Plug_Type                           object
EVSE_ID                            float64
Address_1                           object
City                                object
State/Province                      object
Postal_Code                          int64
Country                             object
Latitude                           float64
Longitude  

In [17]:
#convert date time columns to date time format 

date_cols = ["Start_Date", "End_Date", "Transaction_Date_(Pacific_Time)"]
duration_cols = ["Total_Duration_(hh:mm:ss)", "Charging_Time_(hh:mm:ss)"]

for col in date_cols:
     data[date_cols] = data[date_cols].apply(pd.to_datetime, errors = "coerce")


for col in duration_cols:
    data[duration_cols] = data[duration_cols].apply(pd.to_timedelta, errors="coerce")
    


In [18]:
#replace missing EVSE ID, County, System S/N and Model Number, Driver Postal Code, User ID values with 0. These data were collected
#later in the data cllection process, and were not collected at the beginning of the dataset

data[['EVSE_ID', 'County', 'System_S/N', 'Model_Number', 'Driver_Postal_Code', 'User_ID']] = data[['EVSE_ID', 'County', 'System_S/N', 'Model_Number', 'Driver_Postal_Code', 'User_ID']].fillna(0)

#replace missing currency values with 'USD'
#all transactions occured in Palo Alto, California, USA and USD currency was used.

data['Currency'] = data['Currency'].fillna("USD")

#replace NAN values for ended by column with unknown.

data['Ended_By'] = data['Ended_By'].fillna("Unknown")

#replace NAN values for port time column with unknown.

data['Port_Type'] = data['Port_Type'].fillna("Unknown")

#replace NAN values in transaction date with the Start Date value column

data['Transaction_Date_(Pacific_Time)'] = data['Transaction_Date_(Pacific_Time)'].fillna(data['Start_Date'])

#replace end date NAN with start date + duration time 

data['End_Date'] = data['End_Date'].fillna((data['Start_Date']+data['Total_Duration_(hh:mm:ss)']))

In [19]:
#check for nan

null_data = data[data.isnull().any(axis=1)]
null_data

,Station_Name,MAC_Address,Org_Name,Start_Date,Start_Time_Zone,End_Date,End_Time_Zone,Transaction_Date_(Pacific_Time),Total_Duration_(hh:mm:ss),Charging_Time_(hh:mm:ss),Energy_(kWh),GHG_Savings_(kg),Gasoline_Savings_(gallons),Port_Type,Port_Number,Plug_Type,EVSE_ID,Address_1,City,State/Province,Postal_Code,Country,Latitude,Longitude,Currency,Fee,Ended_By,Plug_In_Event_Id,Driver_Postal_Code,User_ID,County,System_S/N,Model_Number


In [20]:
#create dataframe of 2019 data 
mask_2019 = (data['End_Date'] > "1/1/2019") & (data["End_Date"] <= "1/1/2020")
data_2019 = data[mask_2019].reset_index(drop=True)
data_2019.head()

,Station_Name,MAC_Address,Org_Name,Start_Date,Start_Time_Zone,End_Date,End_Time_Zone,Transaction_Date_(Pacific_Time),Total_Duration_(hh:mm:ss),Charging_Time_(hh:mm:ss),Energy_(kWh),GHG_Savings_(kg),Gasoline_Savings_(gallons),Port_Type,Port_Number,Plug_Type,EVSE_ID,Address_1,City,State/Province,Postal_Code,Country,Latitude,Longitude,Currency,Fee,Ended_By,Plug_In_Event_Id,Driver_Postal_Code,User_ID,County,System_S/N,Model_Number
0,PALO ALTO CA / RINCONADA LIB 3,000D:6F00:015A:96E4,City of Palo Alto,2018-12-31 13:40:00,PST,2019-01-01 00:34:00,PST,2019-01-01 00:34:00,0 days 10:54:02,0 days 09:13:29,55.790,23.432,7.002,Level 2,2,J1772,43583.0,1213 Newell Rd,Palo Alto,California,94303,United States,37.445496,-122.138924,USD,12.83,Plug Out at Vehicle,1546292391,94303.0,2.35396e+06,San Mateo County,1.152100e+11,CTHDR-S
1,PALO ALTO CA / WEBSTER #1,0024:B100:0002:4233,City of Palo Alto,2018-12-31 19:04:00,PST,2019-01-01 00:39:00,PST,2019-01-01 00:40:00,0 days 05:34:35,0 days 04:42:51,15.499,6.510,1.945,Level 2,2,J1772,104427.0,520 Webster St,Palo Alto,California,94301,United States,37.448284,-122.158272,USD,4.04,Plug Out at Vehicle,3565,94020.0,335055,Santa Clara County,1.336410e+11,CT4020-HD-GW
2,PALO ALTO CA / HIGH #1,0024:B100:0002:1262,City of Palo Alto,2018-12-31 20:56:00,PST,2019-01-01 00:56:00,PST,2019-01-01 00:57:00,0 days 04:00:07,0 days 03:17:12,8.918,3.746,1.119,Level 2,2,J1772,104339.0,528 High St,Palo Alto,California,94301,United States,37.443211,-122.163078,USD,2.83,Plug Out at Vehicle,1652,94539.0,3.47049e+06,Santa Clara County,1.425410e+11,CT4020-HD-GW
3,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,City of Palo Alto,2018-12-31 21:04:00,PST,2019-01-01 04:12:00,PST,2019-01-01 04:13:00,0 days 07:08:16,0 days 03:29:30,15.626,6.563,1.961,Level 2,2,J1772,191815.0,445 Bryant St,Palo Alto,California,94301,United States,37.446690,-122.162231,USD,9.86,Plug Out at Vehicle,2791,94403.0,681163,Santa Clara County,1.707410e+11,CT4020-HD
4,PALO ALTO CA / HAMILTON #2,0024:B100:0002:965D,City of Palo Alto,2018-12-31 21:15:00,PST,2019-01-01 03:37:00,PST,2019-01-01 03:38:00,0 days 06:22:17,0 days 06:21:59,35.793,15.033,4.492,Level 2,2,J1772,109785.0,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444706,-122.160324,USD,8.23,Plug Out at Vehicle,4014,94302.0,2.51748e+06,Santa Clara County,1.739410e+11,CT4020-HD-GW


In [21]:
#convert time delta columns to hours 



data_2019["Total_Duration_hours"] = (data_2019["Total_Duration_(hh:mm:ss)"].dt.total_seconds())/ 3600
data_2019["Charging_Time_hours"] = (data_2019["Charging_Time_(hh:mm:ss)"].dt.total_seconds())/ 3600



In [22]:
data_2019.head()

,Station_Name,MAC_Address,Org_Name,Start_Date,Start_Time_Zone,End_Date,End_Time_Zone,Transaction_Date_(Pacific_Time),Total_Duration_(hh:mm:ss),Charging_Time_(hh:mm:ss),Energy_(kWh),GHG_Savings_(kg),Gasoline_Savings_(gallons),Port_Type,Port_Number,Plug_Type,EVSE_ID,Address_1,City,State/Province,Postal_Code,Country,Latitude,Longitude,Currency,Fee,Ended_By,Plug_In_Event_Id,Driver_Postal_Code,User_ID,County,System_S/N,Model_Number,Total_Duration_hours,Charging_Time_hours
0,PALO ALTO CA / RINCONADA LIB 3,000D:6F00:015A:96E4,City of Palo Alto,2018-12-31 13:40:00,PST,2019-01-01 00:34:00,PST,2019-01-01 00:34:00,0 days 10:54:02,0 days 09:13:29,55.790,23.432,7.002,Level 2,2,J1772,43583.0,1213 Newell Rd,Palo Alto,California,94303,United States,37.445496,-122.138924,USD,12.83,Plug Out at Vehicle,1546292391,94303.0,2.35396e+06,San Mateo County,1.152100e+11,CTHDR-S,10.900556,9.224722
1,PALO ALTO CA / WEBSTER #1,0024:B100:0002:4233,City of Palo Alto,2018-12-31 19:04:00,PST,2019-01-01 00:39:00,PST,2019-01-01 00:40:00,0 days 05:34:35,0 days 04:42:51,15.499,6.510,1.945,Level 2,2,J1772,104427.0,520 Webster St,Palo Alto,California,94301,United States,37.448284,-122.158272,USD,4.04,Plug Out at Vehicle,3565,94020.0,335055,Santa Clara County,1.336410e+11,CT4020-HD-GW,5.576389,4.714167
2,PALO ALTO CA / HIGH #1,0024:B100:0002:1262,City of Palo Alto,2018-12-31 20:56:00,PST,2019-01-01 00:56:00,PST,2019-01-01 00:57:00,0 days 04:00:07,0 days 03:17:12,8.918,3.746,1.119,Level 2,2,J1772,104339.0,528 High St,Palo Alto,California,94301,United States,37.443211,-122.163078,USD,2.83,Plug Out at Vehicle,1652,94539.0,3.47049e+06,Santa Clara County,1.425410e+11,CT4020-HD-GW,4.001944,3.286667
3,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,City of Palo Alto,2018-12-31 21:04:00,PST,2019-01-01 04:12:00,PST,2019-01-01 04:13:00,0 days 07:08:16,0 days 03:29:30,15.626,6.563,1.961,Level 2,2,J1772,191815.0,445 Bryant St,Palo Alto,California,94301,United States,37.446690,-122.162231,USD,9.86,Plug Out at Vehicle,2791,94403.0,681163,Santa Clara County,1.707410e+11,CT4020-HD,7.137778,3.491667
4,PALO ALTO CA / HAMILTON #2,0024:B100:0002:965D,City of Palo Alto,2018-12-31 21:15:00,PST,2019-01-01 03:37:00,PST,2019-01-01 03:38:00,0 days 06:22:17,0 days 06:21:59,35.793,15.033,4.492,Level 2,2,J1772,109785.0,250 Hamilton Ave,Palo Alto,California,94301,United States,37.444706,-122.160324,USD,8.23,Plug Out at Vehicle,4014,94302.0,2.51748e+06,Santa Clara County,1.739410e+11,CT4020-HD-GW,6.371389,6.366389


In [23]:
#save 2019 data for spreadsheet use.

data_2019.to_csv("PA_2019.csv", index=False)